## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shenjiamen,CN,29.9576,122.2980,82.13,78,94,9.80,overcast clouds
1,1,Dikson,RU,73.5069,80.5464,41.95,97,67,8.21,broken clouds
2,2,Rio Gallegos,AR,-51.6226,-69.2181,37.45,65,75,10.36,broken clouds
3,3,Kuryk,KZ,43.2000,51.6500,89.28,30,87,12.12,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.64,78,16,11.61,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 99


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shenjiamen,CN,29.9576,122.2980,82.13,78,94,9.80,overcast clouds
3,3,Kuryk,KZ,43.2000,51.6500,89.28,30,87,12.12,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.64,78,16,11.61,few clouds
6,6,Alofi,NU,-19.0595,-169.9187,78.69,89,98,8.05,light rain
7,7,Puerto Carreno,CO,6.1890,-67.4859,77.86,80,100,4.16,overcast clouds
8,8,Harper,LR,4.3750,-7.7169,75.49,86,99,10.56,light rain
9,9,Carnarvon,AU,-24.8667,113.6333,71.67,62,23,11.50,few clouds
10,10,Touros,BR,-5.1989,-35.4608,76.10,88,38,12.26,scattered clouds
12,12,Bloomfield,US,40.8068,-74.1854,70.65,90,90,1.01,mist
15,15,Poum,NC,-20.2333,164.0167,71.22,57,6,14.72,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                388
City                   388
Country                388
Lat                    388
Lng                    388
Max Temp               388
Humidity               388
Cloudiness             388
Wind Speed             388
Current Description    388
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shenjiamen,CN,29.9576,122.2980,82.13,78,94,9.80,overcast clouds
3,3,Kuryk,KZ,43.2000,51.6500,89.28,30,87,12.12,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.64,78,16,11.61,few clouds
6,6,Alofi,NU,-19.0595,-169.9187,78.69,89,98,8.05,light rain
7,7,Puerto Carreno,CO,6.1890,-67.4859,77.86,80,100,4.16,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shenjiamen,CN,82.13,overcast clouds,29.9576,122.2980,
3,Kuryk,KZ,89.28,overcast clouds,43.2000,51.6500,
4,Rikitea,PF,74.64,few clouds,-23.1203,-134.9692,
6,Alofi,NU,78.69,light rain,-19.0595,-169.9187,
7,Puerto Carreno,CO,77.86,overcast clouds,6.1890,-67.4859,
8,Harper,LR,75.49,light rain,4.3750,-7.7169,
9,Carnarvon,AU,71.67,few clouds,-24.8667,113.6333,
10,Touros,BR,76.10,scattered clouds,-5.1989,-35.4608,
12,Bloomfield,US,70.65,mist,40.8068,-74.1854,
15,Poum,NC,71.22,clear sky,-20.2333,164.0167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shenjiamen,CN,82.13,overcast clouds,29.9576,122.2980,Jinjiang Inn Zhoushan Putuo Shenjiamen
3,Kuryk,KZ,89.28,overcast clouds,43.2000,51.6500,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...
4,Rikitea,PF,74.64,few clouds,-23.1203,-134.9692,People ThankYou
6,Alofi,NU,78.69,light rain,-19.0595,-169.9187,Taloa Heights
7,Puerto Carreno,CO,77.86,overcast clouds,6.1890,-67.4859,Casa Hotel Victoria
8,Harper,LR,75.49,light rain,4.3750,-7.7169,Screensaver
9,Carnarvon,AU,71.67,few clouds,-24.8667,113.6333,Hospitality Carnarvon
10,Touros,BR,76.10,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
12,Bloomfield,US,70.65,mist,40.8068,-74.1854,La Quinta Inn & Suites by Wyndham Clifton/Ruth...
15,Poum,NC,71.22,clear sky,-20.2333,164.0167,Gîte kajeon


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   388
Country                388
Max Temp               388
Current Description    388
Lat                    388
Lng                    388
Hotel Name             388
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))